In [1]:
import sys
import psycopg2
import pandas.io.sql as psql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error

import my_pass

In [2]:
def reading(host, DB, username, password):
    conn_string = "host=" + host + " dbname=" + DB + " user=" + username + " password=" + password
    # get connected to the database
    connection = psycopg2.connect(conn_string)
    query = "SELECT * FROM b2w_schema.sales_agg;"
    sales = psql.read_sql(query, connection)
    # Here I'm only selecting immeadiate pay
    query = "SELECT prod_id,date_order,competitor, min(competitor_price) as competitor_price\
            FROM b2w_schema.comp_prices \
            GROUP BY prod_id,date_order,competitor,pay_type \
            HAVING pay_type = 2;"
    price_im = psql.read_sql(query, connection)
    return (sales,price_im)

In [3]:
class ml_models(object):

    def __init__(self, data, product):
        self.df = data
        self.product = product

    def prepare_data(self):
        # selecting a specific product to analyse
        self.df = self.df[self.df['prod_id'] == self.product]

        # Remove columns that only have missing values
        self.df = self.df.dropna(axis=1,how='all')

        # Transforming categorical varibales into factors
        le_day_week = preprocessing.LabelEncoder()
        self.df['day_week'] = le_day_week.fit_transform(self.df['day_week'])
        self.df['month'] = le_day_week.fit_transform(self.df['month'])

    def impute_values(self, cols, strategy = 'mean', axis = 0):

        imp = Imputer(missing_values='NaN', strategy = strategy, axis = axis)
        imp.fit(self.df[cols])
        self.df[cols] = imp.transform(self.df[cols])

    def price_diff(self, comp_price = ['C1','C2','C3','C4','C5','C6']):
        self.df['C1'] = self.df['price'] - self.df['C1']
        self.df['C2'] = self.df['price'] - self.df['C2']
        self.df['C3'] = self.df['price'] - self.df['C3']
        self.df['C4'] = self.df['price'] - self.df['C4']
        self.df['C5'] = self.df['price'] - self.df['C5']
        self.df['C6'] = self.df['price'] - self.df['C6']

    def make_train_test(self,frac = 0.8, random_state = 200):
        # creating a training and test sets
        self.df_train = self.df.sample(frac = frac, random_state = random_state)
        self.df_test = self.df.drop(self.df_train.index)

    def select_features(self, keep_feat = [], drop_feat = []):

        # Isolate Response variable
        self.Y_train, self.Y_test = self.df_train['qty_order'], self.df_test['qty_order']

        # warning: drop_feat overrides keep_feat !!!
        if drop_feat:
            self.X_train = self.df_train.drop(drop_feat, axis=1, inplace=False)
            self.X_test = self.df_test.drop(drop_feat, axis=1, inplace=False)
        elif keep_feat:
            self.X_train = self.df_train[keep_feat]
            self.X_test = self.df_test[keep_feat]

        # Remove Response variable from regressor set
        self.X_train.drop('qty_order', axis=1, inplace=True)
        self.X_test.drop('qty_order', axis=1, inplace=True)

        # Total number of regressors
        self.n_regressors = len(self.X_test.columns)

    def predict_test(self, print_mse = True):
        # Predict and update dataset
        self.Y_pred = self.clf.predict(self.X_test)
        mse = mean_squared_error(self.Y_test, self.Y_pred)
        if print_mse:
            print "MSE: %.4f" % mean_squared_error(self.Y_test, self.Y_pred)

In [4]:
class GBM(ml_models):
    # Gradient Boosting
    def __init__(self, data, product):
        ml_models.__init__(self, data, product)

    def fit_gb(self, params):
        # Fit model
        print self.X_train.columns
        self.clf = GradientBoostingRegressor(**params)
        self.clf.fit(self.X_train, self.Y_train)

    def plot_feature_importance(self, n):
        importances = self.clf.feature_importances_
        feature_names = self.X_test.columns
        indices = np.argsort(importances)[::-1][:n]
        fig, ax = plt.subplots(1,1)
        fig.set_size_inches(10,6)
        plt.title("Feature importances", fontsize = 16)
        xlabels = [feature_names[int(i)] for i in indices]
        plt.bar(range(n), importances[indices],
                color="#799DBB",  align="center")
        plt.grid()
        plt.xticks(range(n), xlabels, rotation=90)
        plt.xlim([-1, n])
        plt.ylim([0, min(1, max(importances[indices]+0.0005))])
        plt.xlabel('Features', fontsize = 14)
        plt.ylabel('Feature Importance', fontsize = 14)
        plt.title('Product '+  self.product +' Variable Importance')
        plt.show()

In [5]:
class MLR(ml_models):
    # Multiple linear regression
    def __init__(self, data, product):
        ml_models.__init__(self, data, product)

    def fit_mlr(self):
        # Fit model
        self.clf = linear_model.LinearRegression(fit_intercept=False)
        self.clf.fit(self.X_train, self.Y_train)
        print self.clf.coef_

# Analysis

## Reading Data

In [6]:
# User inputs
host = 'localhost'
DB = 'postgres'
username = my_pass.username
password = my_pass.password

# Reading tables
sales, price_im = reading(host, DB, username, password)

## Feature Engineering

In [19]:
# Reshaping the data to perform a join in order to get Competitor's Prices
price_im_wide = pd.pivot_table(price_im, index = ['prod_id','date_order'], columns = ['competitor'], values = 'competitor_price')
price_im_wide.reset_index(inplace = True)
df = pd.merge(sales, price_im_wide, how='left', on=['prod_id','date_order'])

# Reshaping the data to perform a join in order to get the Product's Prices
day_price_prod_wide = pd.pivot_table(day_price_prod, index = ['date_order'], columns = ['prod_id'], values = 'price')
day_price_prod_wide.reset_index(inplace = True)
df = pd.merge(df, day_price_prod_wide, how='left', on=['date_order'])

# Quantity sold from previous day
df['qty_lag_1'] = df['qty_order'].shift(1)
df.loc[0,('qty_lag_1')] = df.loc[1,('qty_lag_1')]

# Cumsum of Past 3 days
# To-do

## Fitting Models

In [20]:
gb_p2 = GBM(df, 'P2') 
gb_p2.prepare_data() 
gb_p2.impute_values(cols = ['C1','C2','C3','C4','C5','C6'])
gb_p2.price_diff()
gb_p2.make_train_test()

In [21]:
gb_p2.df_train.head(4)

,prod_id,qty_order,revenue,price,date_order,day_week,month,C1,C2,C3,...,P1,P2,P3,P4,P5,P6,P7,P8,P9,qty_lag_1
1120,P2,565,366733.52,649.085876,2015-07-30,4,6,-14.914124,64.175876,18.235876,...,1484.946875,649.085876,1170.650,540.55,932.326667,1708.582500,733.205388,415.089462,439.000000,19
793,P2,229,161808.24,706.586201,2015-04-22,6,3,77.486201,77.486201,7.586201,...,1421.736364,706.586201,1300.075,561.35,1130.756000,1782.798750,823.832817,490.962500,504.677000,734
2106,P2,79,59139.94,748.606835,2015-03-23,1,2,89.606835,89.606835,89.606835,...,1418.258333,748.606835,1426.500,NaN,NaN,1748.691538,846.550962,591.512500,581.885714,4
1973,P2,45,38892.19,864.270889,2015-02-13,0,1,65.270889,65.270889,86.270889,...,1474.007143,864.270889,1499.000,NaN,NaN,2439.000000,827.828393,NaN,NaN,90


In [22]:
price_im_wide.head(4)

competitor,prod_id,date_order,C1,C2,C3,C4,C5,C6
0,P1,2015-03-15,NaN,NaN,NaN,NaN,1499.00,1499
1,P1,2015-03-16,NaN,1362.5,NaN,NaN,1424.05,1499
2,P1,2015-03-17,1362.5,1362.5,1362.5,NaN,1424.05,1499
3,P1,2015-03-18,1362.5,NaN,1362.5,NaN,1424.05,1499


In [25]:
price_im_wide.sort(['prod_id', 'date_order'], ascending=[1, 1], inplace = True)
price_im_wide.head(4)

competitor,prod_id,date_order,C1,C2,C3,C4,C5,C6
0,P1,2015-03-15,NaN,NaN,NaN,NaN,1499.00,1499
1,P1,2015-03-16,NaN,1362.5,NaN,NaN,1424.05,1499
2,P1,2015-03-17,1362.5,1362.5,1362.5,NaN,1424.05,1499
3,P1,2015-03-18,1362.5,NaN,1362.5,NaN,1424.05,1499


In [ ]:
gb_p2.select_features(drop_feat = ['prod_id','revenue','date_order'])
params = {'n_estimators': 1000, 'max_depth': 2, 'max_features': 'sqrt', 'random_state': 5}
gb_p2.fit_gb(params)

# Predict
gb_p2.predict_test()
d = {'price': gb_p2.X_test['price'], 'Y_test': gb_p2.Y_test, 'Y_pred': gb_p2.Y_pred}
ans_df = pd.DataFrame(data=d)
ans_df.head(10)